In [ ]:
# opengrid imports
from opengrid.library import misc, houseprint, caching
from opengrid.library.analysis import DailyAgg
from opengrid import config
from opengrid.library.slack import Slack
from opengrid.library import alerts
c=config.Config()

# other imports
import pandas as pd
import json
import charts
import numpy as np
import os
import datetime as dt
import pytz
BXL = pytz.timezone('Europe/Brussels')


# configuration for the plots
DEV = c.get('env', 'type') == 'dev' # DEV is True if we are in development environment, False if on the droplet
print("Environment configured for development: {}".format(DEV))
if not DEV:
    # production environment: don't try to display plots
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.dates import MinuteLocator, HourLocator, DateFormatter, AutoDateLocator, num2date

if DEV:
    if c.get('env', 'plots') == 'inline':
        %matplotlib inline
    else:
        %matplotlib qt
else:
    pass # don't try to render plots
plt.rcParams['figure.figsize'] = 12,8

In [ ]:
hp = houseprint.Houseprint()
sensors = hp.get_sensors(sensortype='electricity') # sensor objects

# Remove some sensors
exclude = [
            '565de0a7dc64d8370aa321491217b85f' # 3E
          ]
solar = [x.key for x in hp.search_sensors(type='electricity', system='solar')]
exclude += solar

for s in sensors:
    if s.key in exclude:
        sensors.remove(s)

hp.init_tmpo()

In [ ]:
hp.sync_tmpos()

In [ ]:
dt_now = pd.Timestamp('now', tz='UTC')
dt_start = dt_now - pd.Timedelta(hours=24)
df = hp.get_data(sensors=sensors, head=dt_start)

In [ ]:
df_total = pd.DataFrame(df.sum()/60/1000)
df_total.columns=['sum']

Sort the daily consumptions

In [ ]:
df_total.sort(columns=['sum'], axis=0, inplace=True)
df_total['percentile'] = np.array(range(len(df_total)))/float(len(df_total))

In [ ]:
df_total

# Setup percentile slack bot

In [ ]:
slack_url = c.get('Slack', 'webhook')
username = 'PercentileBot'
channel = "junk" # we don't want to clutter up everything
emoji = ':warning:'
slack = Slack(url=slack_url, username=username, channel=channel, emoji=emoji)

# Setup absolute slack bot

In [ ]:
slack_url = c.get('Slack', 'webhook')
username = 'AbsoluteBot'
channel = "junk" # we don't want to clutter up everything
emoji = ':warning:'
slack = Slack(url=slack_url, username=username, channel=channel, emoji=emoji)

# Create the alerts and send

In [ ]:
alerts.create_alerts(df_total, hp, 'percentile_analysis', slack, column='percentile')
#alerts.create_alerts(df_total, hp, 'absolute_analysis', slack, column='sum')

# Attention, run this only once, to create a default alerts.cfg

In [ ]:
df_total

In [ ]:
df['ad322ec64956ed232cb94a72517004ab'].plot()

In [ ]:
fig = plt.gcf()

In [ ]:
fig.savefig()